In [1]:
import utils.SampleDataRetriever as SampleDataRetriever
import time

In [2]:
text = SampleDataRetriever.retrieve_rojin_to_umi()

Extracted file:  <ZipInfo filename='rojinto_umi.txt' compress_type=deflate external_attr=0x20 file_size=122222 compress_size=54462>


In [3]:
from gensim.models import word2vec
from janome.tokenizer import Tokenizer

In [4]:
t = Tokenizer()

# wakati書きを使う

In [5]:
wakati_1= t.tokenize(text, wakati=True)

In [6]:
wakati_1

['彼',
 'は',
 '老い',
 'て',
 'い',
 'た',
 '。',
 '小さな',
 '船',
 'で',
 'メキシコ',
 '湾流',
 'に',
 '漕ぎ',
 '出し',
 '、',
 '独り',
 'で',
 '漁',
 'を',
 'し',
 'て',
 'い',
 'た',
 '。',
 '一',
 '匹',
 'も',
 '釣れ',
 'ない',
 '日',
 'が',
 '、',
 '既に',
 '八',
 '四',
 '日',
 'も',
 '続い',
 'て',
 'い',
 'た',
 '。',
 '最初',
 'の',
 '四',
 '〇',
 '日',
 'は',
 '少年',
 'と',
 '一緒',
 'だっ',
 'た',
 '。',
 'しかし',
 '、',
 '獲物',
 'の',
 '無い',
 'まま',
 'に',
 '四',
 '〇',
 '日',
 'が',
 '過ぎる',
 'と',
 '、',
 '少年',
 'に',
 '両親',
 'が',
 '告げ',
 'た',
 '。',
 'あの',
 '老人',
 'は',
 'もう',
 '完全',
 'に',
 '「',
 'サラオ',
 '」',
 'な',
 'ん',
 'だ',
 'よ',
 '、',
 'と',
 '。',
 'サラオ',
 'と',
 'は',
 '、',
 'すっかり',
 '運',
 'に',
 '見放さ',
 'れ',
 'た',
 'という',
 'こと',
 'だ',
 '。',
 '少年',
 'は',
 '両親',
 'の',
 '言いつけ',
 '通り',
 'に',
 '別',
 'の',
 'ボート',
 'に',
 '乗り換え',
 '、',
 '一',
 '週間',
 'で',
 '三',
 '匹',
 'も',
 '立派',
 'な',
 '魚',
 'を',
 '釣り上げ',
 'た',
 '。',
 '老人',
 'が',
 '毎日',
 '空っぽ',
 'の',
 '船',
 'で',
 '帰っ',
 'て',
 'くる',
 'の',
 'を',
 '見る',
 'たび',
 'に',
 '、',
 '少年',
 'の',
 '心',
 'は',
 '痛ん',
 'だ',
 '。',


## OKのケース
Wakati書きをリストにWrapしてWord2Vecの学習データとしてINPUT

In [18]:
# 　https://stackoverflow.com/questions/45420466/gensim-keyerror-word-not-in-vocabulary
#  model = word2vec.Word2Vec(wakati_1, size=100, min_count=1, window=3, iter=3) <-- wakati_1はiterative of sentencesではない!!

time_s  = time.time()
model = word2vec.Word2Vec([wakati_1], size=100, min_count=1, window=3, iter=3)
time_e  = time.time()
print(time_e - time_s)
print('--')

results = model.wv.most_similar(positive="彼", topn=3)
for result in results:
    print(result[0],result[1])
    
print("===")

# # # メキシコがVocabularyになくErrorになる ###
try:
    results = model.wv.most_similar(positive="メキシコ", topn=3)
    for result in results:
        print(result[0],result[1])
except KeyError as error:
        print("Error: ",error)

0.13224101066589355
--
。 0.9996094703674316
の 0.9995978474617004
に 0.9995735883712769
===
詰まっ 0.7078825235366821
作業 0.7037894129753113
こんなに 0.7021613121032715


## ダメなケース
　Word2Vec学習のデータとして、Wakati書きされた単語で個々のword_listを使う。この場合Wort to Vecではコンテキストが取れないので、あまり意味が無い学習になっている。普通こういうのはやらない。

In [8]:
#対象の品詞
obj_list = ['名詞', '動詞','形容詞']
def extract_words(s):
#     print('===')
#     print(s)
    tokens = t.tokenize(s)
    r = [token.base_form for token in tokens if token.part_of_speech.split(',')[0] in obj_list]
    return r

word_list = [extract_words(sentence) for sentence in wakati_1]

model3 = word2vec.Word2Vec(word_list, size=100, min_count=1, window=3, iter=3)

# results
results = model3.wv.most_similar(positive="彼", topn=3)
for result in results:
    print(result[0],result[1])
print('--')

results = model3.wv.most_similar(positive="メキシコ", topn=3)
for result in results:
    print(result[0],result[1])
    
print(word_list)

注意 0.30049434304237366
酒屋 0.29441943764686584
テラス 0.2923293709754944
--
携える 0.3298115134239197
釣り場 0.2912035286426544
動ける 0.28831031918525696
[['彼'], [], ['老い'], [], ['いる'], [], [], [], ['船'], [], ['メキシコ'], ['湾流'], [], ['漕ぐ'], ['出す'], [], ['独り'], [], ['漁'], [], ['する'], [], ['いる'], [], [], ['一'], ['匹'], [], ['釣る'], ['ない'], ['日'], [], [], [], ['八'], ['四'], ['日'], [], ['続く'], [], ['いる'], [], [], ['最初'], [], ['四'], [], ['日'], [], ['少年'], [], ['一緒'], [], [], [], [], [], ['獲物'], [], ['無い'], ['まま'], [], ['四'], [], ['日'], [], ['過ぎる'], [], [], ['少年'], [], ['両親'], [], ['告げ'], [], [], [], ['老人'], [], [], ['完全'], [], [], ['サラオ'], [], [], [], [], [], [], [], [], ['サラオ'], [], [], [], [], ['運'], [], ['見放す'], ['れる'], [], [], ['こと'], [], [], ['少年'], [], ['両親'], [], ['言いつける'], ['通り'], [], ['別'], [], ['ボート'], [], ['乗り換え'], [], ['一'], ['週間'], [], ['三'], ['匹'], [], ['立派'], [], ['魚'], [], ['釣り上げる'], [], [], ['老人'], [], ['毎日'], ['空っぽ'], [], ['船'], [], ['帰る'], [], ['くる'], [], [], ['見る'], ['たび'], [], [], ['少年'

# 　。でSplitしたSentencesを使う

In [9]:
sentences = text.split('。')
sentences

['\n\u3000彼は老いていた',
 '小さな船でメキシコ湾流に漕ぎ出し、独りで漁をしていた',
 '一匹も釣れない日が、既に八四日も続いていた',
 '最初の四〇日は少年と一緒だった',
 'しかし、獲物の無いままに四〇日が過ぎると、少年に両親が告げた',
 'あの老人はもう完全に「サラオ」なんだよ、と',
 'サラオとは、すっかり運に見放されたということだ',
 '少年は両親の言いつけ通りに別のボートに乗り換え、一週間で三匹も立派な魚を釣り上げた',
 '老人が毎日空っぽの船で帰ってくるのを見るたびに、少年の心は痛んだ',
 '彼はいつも老人を迎えに行って、巻いたロープ、手鉤、銛、帆を巻きつけたマストなどを運ぶ手伝いをするのだった',
 '粉袋で継ぎあてされた帆は、巻き上げられて、永遠の敗北を示す旗印のように見えた',
 '\n\u3000老人は細くやつれ、首筋には深い皺が刻まれていた',
 '頬には、熱帯の海に反射した日光によって、まるで皮膚癌のような褐色のしみができていた',
 'しみは顔の両側に首近くまで連なっている',
 '両手には、大きな魚の食らいついたロープを制する時にできた、深い傷痕がいくつもあった',
 'しかし傷痕は新しいものではない',
 '魚などいない砂漠、風に侵食された砂漠のように、古い傷痕だった',
 '\n\u3000彼に関しては何もかもが古かった',
 'ただ、その両眼を除いては',
 '彼の眼は、海と同じ色に輝き、喜びと不屈の光をたたえていた',
 '\n「サンチャゴ」少年は、船を着けた岸の斜面をのぼりながら老人に呼びかけた',
 '「また一緒に行きたいな',
 'お金も多少貯まったし」\n\u3000老人は少年に漁を教えてきた',
 '少年は彼を慕っていた',
 '\n「だめだ」老人は言った',
 '「お前の船はついてる',
 '仲間を変えないほうがいい」\n「でも僕らは前に、八七日も不漁だった後で、三週間毎日大物を釣ったことがあったじゃないか」\n「あったな」老人は言った',
 '「分かってるさ',
 'お前が船を変えたのは、俺の腕を疑ったからじゃない」\n「親父だよ、船を変えさせたのは',
 '僕は子供だから、従うしかないんだ」\n「分かってる」老人は言った',
 '「当然のことだ」\n「親父には、信じるってことができな

## ダメなケース
sentencesをそのまま学習 (INPUTが単語リストになってないので、このパターンは普通やらない！！）

In [10]:
# word2vecは単語に分けて学習させるので、sentencesのままは普通INPUTしない。
model2 = word2vec.Word2Vec(sentences, size=100, min_count=1, window=3, iter=3)
results = model2.wv.most_similar(positive="彼", topn=5)
for result in results:
    print(result[0],result[1])

# # # メキシコがVocabularyになくErrorになる ###
# results = model2.wv.most_similar(positive="メキシコ", topn=5)
# for result in results:
#     print(result[0],result[1])

そ 0.9996001124382019
俺 0.9995911121368408
は 0.9995648860931396
に 0.9995532631874084
海 0.9995461106300354


## OKのケース　Word2Vec学習のデータとして、品詞（名詞、動詞、形容詞）のみのWord listを使う

In [21]:
#対象の品詞
# obj_list = ['名詞', '動詞','形容詞','形容動詞','副詞','連体詞','接続詞','感動詞', '助動詞', '助詞']
obj_list = ['名詞', '動詞','形容詞']
def extract_words(s):
    tokens = t.tokenize(s)
    r = [token.base_form for token in tokens if token.part_of_speech.split(',')[0] in obj_list]
    # https://www.excellence-blog.com/2018/05/06/word2vec-%E5%8D%98%E8%AA%9E%E3%81%AE%E3%83%99%E3%82%AF%E3%83%88%E3%83%AB%E8%A1%A8%E7%8F%BE/
    # 
    return r

word_list = [extract_words(sentence) for sentence in sentences]

time_s = time.time()
model3 = word2vec.Word2Vec(word_list, size=100, min_count=1, window=3, iter=3)
time_e  = time.time()
print(time_e - time_s)
print('--')

# results
results = model3.wv.most_similar(positive="彼", topn=3)
for result in results:
    print(result[0],result[1])
print('--')

results = model3.wv.most_similar(positive="メキシコ", topn=3)
for result in results:
    print(result[0],result[1])
print("---")
print(word_list)

0.1109919548034668
--
する 0.7388802766799927
の 0.7172468900680542
いる 0.7126013040542603
--
携える 0.33173948526382446
動ける 0.2912191152572632
釣り場 0.29086220264434814
---
[['彼', '老いる', 'いる'], ['船', 'メキシコ', '湾流', '漕ぐ', '出す', '独り', '漁', 'する', 'いる'], ['一', '匹', '釣れる', '日', '八', '四', '日', '続く', 'いる'], ['最初', '四', '〇', '日', '少年', '一緒'], ['獲物', '無い', 'まま', '四', '〇', '日', '過ぎる', '少年', '両親', '告げる'], ['老人', '完全', 'サラオ', 'ん'], ['サラオ', '運', '見放す', 'れる', 'こと'], ['少年', '両親', '言いつける', '通り', '別', 'ボート', '乗り換える', '一', '週間', '三', '匹', '立派', '魚', '釣り上げる'], ['老人', '毎日', '空っぽ', '船', '帰る', 'くる', 'の', '見る', 'たび', '少年', '心', '痛む'], ['彼', '老人', '迎える', '行く', '巻く', 'ロープ', '手鉤', '銛', '帆', '巻く', 'つける', 'マスト', '運ぶ', '手伝い', 'する', 'の'], ['粉', '袋', '継ぎ', 'あて', 'する', 'れる', '帆', '巻き上げる', 'られる', '永遠', '敗北', '示す', '旗印', 'よう', '見える'], ['老人', '細い', 'やつれる', '首筋', '深い', '皺', '刻む', 'れる', 'いる'], ['頬', '熱帯', '海', '反射', 'する', '日光', '皮膚', '癌', 'よう', '褐色', 'しみ', 'できる', 'いる'], ['しみ', '顔', '両側', '首', '近い', '連なる', 'いる'], ['両手', '魚', '食らいつく

# Parameters

In [ ]:
# https://www.kaggle.com/pierremegret/gensim-word2vec-tutorial#The-parameters:
# https://qiita.com/mergit/items/822dc49343c887019d44 
model3 = word2vec.Word2Vec(word_list, size=100, min_count=1, window=3, iter=3)
# min_count : この数より小さい出現頻度の単語は除外する
# window : windowサイズ(ターゲットの単語の前後の単語数。ゼロから始めるDL　Vol2のW2Vのコンテキストという説明を参照)
# size : 単語ベクトルの次元数